In [1]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

solvers.options['show_progress'] = False
solvers.options['abstol'] = 1e-12
solvers.options['reltol'] = 1e-12
solvers.options['feastol'] = 1e-12

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

In [13]:
for subdir, dirs, files in os.walk("./RUNS"):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".txt.merged"):
            if not ".ipynb_checkpoints" in filepath:
                print (filepath)
                os.rename(filepath, filepath.replace(".txt.merged", ".txt"))

./RUNS/CAL/C_1EPS01/300/optimal_values.txt.merged
./RUNS/CAL/C_1EPS01/400/optimal_values.txt.merged
./RUNS/CAL/C_1EPS01/100/optimal_values.txt.merged
./RUNS/CAL/C_1EPS01/500/optimal_values.txt.merged
./RUNS/CAL/C_1EPS01/200/optimal_values.txt.merged
./RUNS/CAL/C_1EPS01/600/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/300/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/400/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/100/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/500/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/200/optimal_values.txt.merged
./RUNS/CAL/C1EPS_01/600/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/300/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/400/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/100/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/500/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/200/optimal_values.txt.merged
./RUNS/CAL/C_01EPS01/600/optimal_values.txt.merged
./RUNS/CAL/C1EPS_1/300/optimal_values.txt.merged
./RUNS/CAL/C1EPS_1/400/optimal_values.txt.mer

# Parametri default

In [3]:
seed = 123     # random seed
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [4]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1, 0.5, 1, 2, 5, 10]
problem_sizes = [100,200,300,400,500,600]
alphas = [0.1, 0.5, 1, 2, 5]
n_prob = 10

In [5]:
def generate_multiple_alphas(feat, targ, size, alphas, n=n_prob, startdir='./', maxiter=1000):
    """experiments on the alpha paramerer."""
    if not startdir.endswith("/"):
        print("inserting slash in startdir")
        startdir = startdir + "/"
    
    for alpha in alphas:
        alpha_name = str(alpha).replace(".","")
        os.makedirs(f"{startdir}{alpha_name}", exist_ok=True)
        print(f"creating {startdir}{alpha_name}")
    
    for i in range(n):
        K, target = sample_transform_problem(feat, targ, size)
        
        sol = solve_with_cvxopt(K, target, 0.1, 1)['dual objective']
        
        with open(f"{startdir}optimal_values.txt", "a") as f:
            f.write(str(sol) + "\n")
            
        for alpha in alphas:
            print(f"Problem {i} {alpha=}")                    
            ds = DataStorer()
            sg, xg, vg, itg = SLBQP(K, target, 1, 0.1, eps=1e-3, maxIter=maxiter, alpha=alpha, prj_type=GOLDSTEIN, ds=ds)          
            alpha_name = str(alpha).replace(".","")
            ds.save_to_file(f"{startdir}{alpha_name}/{i}.csv")
            
            print(f"\t\t{itg=}")
            

In [6]:
def generate_multiple_epsilons(feat, targ, size, u, epsilons, n, startdir='./'):
    u_name = str(u).replace(".","")
    for e in epsilons:
        eps_name = str(e).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS", exist_ok=True)

    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)
        for e in epsilons:
            print(f"Solving problem {i} with epsilon {e}")
            eps_name = str(e).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS/{i}.csv")
            sol = solve_with_cvxopt(K, target, e, u)['dual objective']
            with open(f"{startdir}C{u_name}EPS_{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol) + "\n")
            
            

In [7]:
def generate_multiple_upperbounds(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    
    eps_name = str(epsilon).replace(".", "")
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
        
    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for u in upper_bounds:
            print(f"Solving problem {i} with U: {u}")
            u_name = str(u).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            
            sol = solve_with_cvxopt(K, target, epsilon, u)['dual objective']
            
            with open(f"{startdir}C_{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol)+"\n")

# ML DATA GENERATION

In [8]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1
ROOT="./RUNS/ML/"

loading from: 
/Users/gerardozinno/repos/ProgettoML/cm/data/ML-CUP19-TR.csv


In [9]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 1
Solving problem 1 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 1 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 1 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 2
Solving problem 2 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 2 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 2 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 3
Solving problem 

In [10]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 0 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 0 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
generating problem 1
Solving problem 1 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 1 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 1 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 1 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 1 with U: 5
	Ab

In [11]:
ml_alpha_startdir = f"RUNS/ML/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, ml_alpha_startdir)

creating RUNS/ML/ALPHA/01
creating RUNS/ML/ALPHA/05
creating RUNS/ML/ALPHA/1
creating RUNS/ML/ALPHA/2
creating RUNS/ML/ALPHA/5
Problem 0 alpha=0.1
		itg=139
Problem 0 alpha=0.5
		itg=104
Problem 0 alpha=1
		itg=238
Problem 0 alpha=2
		itg=1000
Problem 0 alpha=5
		itg=1000
Problem 1 alpha=0.1
		itg=123
Problem 1 alpha=0.5
		itg=124
Problem 1 alpha=1
		itg=325
Problem 1 alpha=2
		itg=1000
Problem 1 alpha=5
		itg=1000
Problem 2 alpha=0.1
		itg=116
Problem 2 alpha=0.5
		itg=125
Problem 2 alpha=1
		itg=582
Problem 2 alpha=2
		itg=1000
Problem 2 alpha=5
		itg=1000
Problem 3 alpha=0.1
		itg=219
Problem 3 alpha=0.5
		itg=140
Problem 3 alpha=1
		itg=278
Problem 3 alpha=2
		itg=1000
Problem 3 alpha=5
		itg=1000
Problem 4 alpha=0.1
		itg=101
Problem 4 alpha=0.5
		itg=142
Problem 4 alpha=1
		itg=375
Problem 4 alpha=2
		itg=1000
Problem 4 alpha=5
		itg=1000
Problem 5 alpha=0.1
		itg=143
Problem 5 alpha=0.5
		itg=116
Problem 5 alpha=1
		itg=247
Problem 5 alpha=2
		itg=1000
Problem 5 alpha=5
		itg=10

# CALIF DATA GENERATION

In [12]:
feat, targ = load_california_dataset()
targ
ROOT="./RUNS/CAL/"

/usr/local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.california_housing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [13]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 1
Solving problem 1 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 1 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 1 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 2
Solving problem 2 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 2 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 2 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 3
Solving problem 

In [14]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 0 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 0 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
generating problem 1
Solving problem 1 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 1 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 1 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 1 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 1 with U: 5
	Ab

In [15]:
cal_alpha_startdir = f"RUNS/CAL/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, cal_alpha_startdir)

creating RUNS/CAL/ALPHA/01
creating RUNS/CAL/ALPHA/05
creating RUNS/CAL/ALPHA/1
creating RUNS/CAL/ALPHA/2
creating RUNS/CAL/ALPHA/5
Problem 0 alpha=0.1
		itg=91
Problem 0 alpha=0.5
		itg=20
Problem 0 alpha=1
		itg=8
Problem 0 alpha=2
		itg=1000
Problem 0 alpha=5
		itg=1000
Problem 1 alpha=0.1
		itg=90
Problem 1 alpha=0.5
		itg=20
Problem 1 alpha=1
		itg=8
Problem 1 alpha=2
		itg=1000
Problem 1 alpha=5
		itg=1000
Problem 2 alpha=0.1
		itg=91
Problem 2 alpha=0.5
		itg=20
Problem 2 alpha=1
		itg=9
Problem 2 alpha=2
		itg=1000
Problem 2 alpha=5
		itg=1000
Problem 3 alpha=0.1
		itg=91
Problem 3 alpha=0.5
		itg=20
Problem 3 alpha=1
		itg=8
Problem 3 alpha=2
		itg=1000
Problem 3 alpha=5
		itg=1000
Problem 4 alpha=0.1
		itg=91
Problem 4 alpha=0.5
		itg=20
Problem 4 alpha=1
		itg=8
Problem 4 alpha=2
		itg=1000
Problem 4 alpha=5
		itg=1000
Problem 5 alpha=0.1
		itg=91
Problem 5 alpha=0.5
		itg=20
Problem 5 alpha=1
		itg=8
Problem 5 alpha=2
		itg=1000
Problem 5 alpha=5
		itg=1000
Problem 6 alpha=

# AIRF DATA GENERATION

In [16]:
feat, targ = load_airfoil_dataset()
targ
ROOT="./RUNS/AIRF/"

/Users/gerardozinno/repos/ProgettoML/cm/data/airfoil_self_noise.csv


In [17]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 1
Solving problem 1 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 1 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 1 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 2
Solving problem 2 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving problem 2 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving problem 2 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
generating problem 3
Solving problem 

In [18]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 0 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 0 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
generating problem 1
Solving problem 1 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving problem 1 with U: 0.5
	About to run GOLDSTEIN with u=0.5
	About to run ROSEN with u=0.5
Solving problem 1 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving problem 1 with U: 2
	About to run GOLDSTEIN with u=2
	About to run ROSEN with u=2
Solving problem 1 with U: 5
	Ab

In [19]:
airf_alpha_startdir = f"RUNS/AIRF/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, airf_alpha_startdir)

creating RUNS/AIRF/ALPHA/01
creating RUNS/AIRF/ALPHA/05
creating RUNS/AIRF/ALPHA/1
creating RUNS/AIRF/ALPHA/2
creating RUNS/AIRF/ALPHA/5
Problem 0 alpha=0.1
		itg=77
Problem 0 alpha=0.5
		itg=19
Problem 0 alpha=1
		itg=10
Problem 0 alpha=2
		itg=1000
Problem 0 alpha=5
		itg=1000
Problem 1 alpha=0.1
		itg=79
Problem 1 alpha=0.5
		itg=19
Problem 1 alpha=1
		itg=13
Problem 1 alpha=2
		itg=1000
Problem 1 alpha=5
		itg=1000
Problem 2 alpha=0.1
		itg=75
Problem 2 alpha=0.5
		itg=19
Problem 2 alpha=1
		itg=13
Problem 2 alpha=2
		itg=1000
Problem 2 alpha=5
		itg=1000
Problem 3 alpha=0.1
		itg=74
Problem 3 alpha=0.5
		itg=20
Problem 3 alpha=1
		itg=10
Problem 3 alpha=2
		itg=1000
Problem 3 alpha=5
		itg=1000
Problem 4 alpha=0.1
		itg=74
Problem 4 alpha=0.5
		itg=20
Problem 4 alpha=1
		itg=11
Problem 4 alpha=2
		itg=1000
Problem 4 alpha=5
		itg=1000
Problem 5 alpha=0.1
		itg=74
Problem 5 alpha=0.5
		itg=20
Problem 5 alpha=1
		itg=11
Problem 5 alpha=2
		itg=1000
Problem 5 alpha=5
		itg=1000
Probl